#### Many To Many

_Many to Many_ adds an __association table between two classes__. The _association table_ is indicated by the `relationship.secondary` argument to `relationship()`. Usually, the _Table_ uses the _MetaData_ object _associated with the declarative base class_, so that the `ForeignKey` directives can __locate the remote tables with which to link__.

In [1]:
from sqlalchemy import Column, ForeignKey, Integer, Table
from sqlalchemy.orm import declarative_base, relationship

In [2]:
Base = declarative_base()

In [3]:
association_table = Table(
    "association_table",
    Base.metadata,
    Column("left_id", ForeignKey("left_table.id")),
    Column("right_id", ForeignKey("right_table.id")),
)

In [4]:
class Parent(Base):
    __tablename__ = "left_table"
    id = Column(Integer, primary_key=True)
    children = relationship("Child", secondary=association_table)

In [5]:
class Child(Base):
    __tablename__ = "right_table"
    id = Column(Integer, primary_key=True)

> ##### Tip
>
> The _"association table"_ above has `foreign key constraints` established that refer to the two entity tables on either side of the relationship. The _data type_ of each of *association.left_id* and *association.right_id* is __normally inferred from that of the referenced table and may be omitted__. It is also __recommended__, though _not in any way required_ by SQLAlchemy, that the _columns which refer to the two entity tables_ are established within either a _unique constraint_ or more commonly as the _primary key constraint_; this ensures that __duplicate rows won't be persisted__ within the table regardless of issues on the application side.

```
association_table = Table(
    "association_table",
    Base.metadata,
    Column("left_id", ForeignKey("left_table.id"), primary_key=True),
    Column("right_id", ForeignKey("right_table.id"), primary_key=True),
)
```

For a `bidirectional relationship`, _both sides_ of the relationship __contain a collection__. Specify using `relationship.back_populates`, and for each `relationship()` specify the _common association table_.

In [6]:
middle_table = Table(
    "middle_table",
    Base.metadata,
    Column("left_id", ForeignKey("left.id"), primary_key=True),
    Column("right_id", ForeignKey("right.id"), primary_key=True),
)

In [7]:
class Left(Base):
    __tablename__ = "left"
    id = Column(Integer, primary_key=True)
    children = relationship("Right", secondary=middle_table, back_populates="parents")

In [8]:
class Right(Base):
    __tablename__ = "right"
    id = Column(Integer, primary_key=True)
    parents = relationship("Left", secondary=middle_table, back_populates="children")

When using the `relationship.backref` parameter instead of `relationship.back_populates`, the _backref_ will __automatically use the same__ `relationship.secondary` argument for the _reverse relationship_.

In [9]:
mid = Table(
    "mid",
    Base.metadata,
    Column("left_id", ForeignKey("lt.id"), primary_key=True),
    Column("right_id", ForeignKey("rt.id"), primary_key=True),
)

In [10]:
class LT(Base):
    __tablename__ = "lt"
    id = Column(Integer, primary_key=True)
    children = relationship("RT", secondary=mid, backref="parents")

In [11]:
class RT(Base):
    __tablename__ = "rt"
    id = Column(Integer, primary_key=True)

The `relationship.secondary` argument of `relationship()` __also accepts a callable that returns the ultimate argument__, which is __evaluated only when mappers are first used__. Using this, we can define the *association_table* at a __later point__, as long as it's available to the callable _after all module initialization is complete_.

```
class Parent(Base):
    __tablename__ = "left_table"
    id = Column(Integer, primary_key=True)
    children = relationship(
        "Child",
        secondary=lambda: association_table,
        backref="parents",
    )
```

With the _declarative extension_ in use, the _traditional "string name of the table"_ is __accepted__ as well, matching the name of the table as stored in `Base.metadata.tables`.

```
class Parent(Base):
    __tablename__ = "left_table"
    id = Column(Integer, primary_key=True)
    children = relationship("Child", secondary="association_table", backref="parents")
```

> ##### Warning
>
> When passed as a _Python-evaluable string_, the `relationship.secondary` argument is interpreted using Python's `eval()` function. __DO NOT PASS UNTRUSTED INPUT TO THIS STRING__.

##### Deleting Rows from the Many to Many Table

A _behavior_ which is __unique__ to the `relationship.secondary` argument to `relationship()` is that the _Table_ which is specified here is __automatically__ subject to _INSERT_ and _DELETE_ statements, as objects are _added or removed from the collection_. There is __no need to delete from this table manually__. The act of removing a record from the collection will have the effect of the row being deleted on flush.

```
# row will be deleted from the "secondary" table automatically
myparent.children.remove(somechild)
```

A question which often arises is how the _row in the "secondary" table_ can be __deleted__ when the _child_ object is __handed directly__ to `Session.delete()`.

```
session.delete(somechild)
```

There are several possibilities here:

* If there is a `relationship()` from Parent to Child, but there is not a _reverse-relationship_ that links a particular Child to each Parent, __SQLAlchemy will not have any awareness__ that when _deleting this particular Child object_, it needs to maintain the `"secondary"` table that _links it to the Parent_. __No delete of the "secondary" table will occur__.

* If there is a _relationship_ that links a particular Child to each Parent, suppose it's called `Child.parents`, SQLAlchemy _by default will load_ in the `Child.parents` collection to _locate all Parent objects_, and __remove each row from the "secondary" table which establishes this link__. Note that this _relationship_ __does not need__ to be `bidirectional`; SQLAlchemy is __strictly__ looking at every `relationship()` associated with the _Child object being deleted_.

* A _higher performing option_ here is to use `ON DELETE CASCADE` directives with the _foreign keys used by the database_. Assuming the database supports this feature, the database itself can be made to __automatically delete rows__ in the `"secondary"` table as referencing rows in `"child"` are deleted. SQLAlchemy can be __instructed to forego actively loading__ in the `Child.parents` collection in this case using the `relationship.passive_deletes` directive on `relationship()`.

Note again, these _behaviors_ are __only relevant__ to the `relationship.secondary` option used with `relationship()`. If dealing with _association tables_ that are __mapped explicitly__ and are __not present__ in the `relationship.secondary` option of a relevant `relationship()`, _cascade rules_ can be _used instead_ to __automatically delete entities__ in reaction to a related entity being deleted.